# Import packages

In [1]:


# -----------------------------------------------------------------------------
# region import packages

# management
import glob
import pickle
import warnings
warnings.filterwarnings('ignore')
import os
import sys  # print(sys.path)
sys.path.append('/work/ollie/qigao001')
os.chdir('/home/users/qino')

# data analysis
import numpy as np
import xarray as xr
import dask
dask.config.set({"array.slicing.split_large_chunks": True})
# from dask.diagnostics import ProgressBar
# pbar = ProgressBar()
# pbar.register()
from scipy import stats
import xesmf as xe
import pandas as pd
from metpy.interpolate import cross_section
from statsmodels.stats import multitest
import pycircstat as circ
from scipy.stats import circstd
import cmip6_preprocessing.preprocessing as cpp

# plot
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import BoundaryNorm, ListedColormap
from matplotlib import cm
import cartopy.crs as ccrs
plt.rcParams['pcolor.shading'] = 'auto'
mpl.rcParams['figure.dpi'] = 600
mpl.rc('font', family='Times New Roman', size=10)
mpl.rcParams['axes.linewidth'] = 0.2
plt.rcParams.update({"mathtext.fontset": "stix"})
import matplotlib.animation as animation
import seaborn as sns
from matplotlib.ticker import AutoMinorLocator

# self defined
from a_basic_analysis.b_module.mapplot import (
    globe_plot,
    hemisphere_plot,
    quick_var_plot,
    mesh2plot,
    framework_plot1,
    remove_trailing_zero,
    remove_trailing_zero_pos,
)

from a_basic_analysis.b_module.basic_calculations import (
    mon_sea_ann,
    find_ilat_ilon,
    regrid,
    time_weighted_mean,
    find_ilat_ilon_general,
)

from a_basic_analysis.b_module.namelist import (
    month,
    month_num,
    month_dec,
    month_dec_num,
    seasons,
    seasons_last_num,
    hours,
    months,
    month_days,
    zerok,
    panel_labels,
    seconds_per_d,
)

from a_basic_analysis.b_module.source_properties import (
    source_properties,
    calc_lon_diff,
)

from a_basic_analysis.b_module.statistics import (
    fdr_control_bh,
    check_normality_3d,
    check_equal_variance_3d,
    ttest_fdr_control,
)

from a_basic_analysis.b_module.component_plot import (
    cplot_ice_cores,
    plt_mesh_pars,
)

# endregion
# -----------------------------------------------------------------------------




# import data

In [2]:
lig_recs = {}

with open('scratch/cmip6/lig/rec/lig_recs_dc.pkl', 'rb') as f:
    lig_recs['DC'] = pickle.load(f)

with open('scratch/cmip6/lig/rec/lig_recs_ec.pkl', 'rb') as f:
    lig_recs['EC'] = pickle.load(f)

with open('scratch/cmip6/lig/rec/lig_recs_jh.pkl', 'rb') as f:
    lig_recs['JH'] = pickle.load(f)

with open('scratch/cmip6/lig/rec/lig_recs_mc.pkl', 'rb') as f:
    lig_recs['MC'] = pickle.load(f)


In [3]:
lig_recs['EC']['SO_ann']

,Station,Latitude,Longitude,Elevation [m],Area,Temperature reconstruction method,Uncertainty on temperature reconstruction [°C],Type,References temperature data,Surface air temperature [°C],...,125 ka Median PIAn [°C],125 ka 2s PIAn [°C],120 ka Median PIAn [°C],120 ka 2s PIAn [°C],115 ka Median PIAn [°C],115 ka 2s PIAn [°C],127-130 ka diff [°C],127-130 ka diff 2s [°C],125-127 ka diff [°C],125-127 ka diff 2s [°C]
27,MD97-2121_B,-40.22,177.59,-3014,Southern Ocean,Alkenone ratio,1.0,Annual SST,"Pahnke et al., 2006",NaN,...,4.90,3.60,4.75,3.66,2.55,3.91,0.66,0.79,-0.26,0.42
33,DSDP-594,-45.31,174.57,-1204,Southern Ocean,Foraminifera transfer function (MAT),0.9,Annual SST,"Wells and Okada, 1997",NaN,...,1.87,2.03,0.11,2.06,-1.56,2.00,-0.51,2.07,-1.01,0.68


In [4]:
lig_recs['JH']['SO_ann']

,Station,Region,Latitude,Longitude,Type,HadISST1.1 (1880-1889) (°C),Interpolated 127 ka value (°C),127 ka SST anomaly (°C),127 ka 2σ (°C)
25,Y9,Southern Ocean,-48.240,177.34,Annual SST,9.61,13.593065,3.983065,4.293972
26,MD97-2108,Southern Ocean,-48.500,149.11,Annual SST,9.50,11.434930,1.934930,4.327741
27,MD97-2109,Southern Ocean,-50.630,169.38,Annual SST,8.90,7.545920,-1.354080,4.809288
28,MD97-2121,Southern Ocean,-40.000,177.00,Annual SST,13.38,20.398419,7.018419,1.658285
29,RC15-61,Southern Ocean,-40.620,-77.20,Annual SST,11.66,7.912209,-3.747791,3.027353
30,ODP-1089,Southern Ocean,-40.940,9.90,Annual SST,11.66,23.162182,11.502182,1.823814
31,ODP-1123,Southern Ocean,-41.790,-171.50,Annual SST,11.70,16.547870,4.847870,4.108847
32,MD73-025,Southern Ocean,-43.820,51.30,Annual SST,11.87,5.078828,-6.791172,1.892125
33,MD97-2106,Southern Ocean,-45.150,146.29,Annual SST,10.86,14.096855,3.236855,4.286759
34,DSDP-594,Southern Ocean,-45.500,174.95,Annual SST,10.86,11.321431,0.461431,3.402782


In [5]:
lig_recs['DC']['annual_128']

,Station,Event,Reference (SST proxy reference),URL ref (SST proxy reference),Reference (d18O reference),URL ref (d18O reference),"Season (A annual, S summer (months JFM))","Proxy (A alkenone UK'37, D diatom as...)",Latitude,Longitude,SST [°C] (Reconstructed),Age [ka BP] (Chronology follows Lisiecki a...),SST anomaly [°C] (Anomaly relative to the World...),HadISST_annual,sst_anom_hadisst_ann
0,DSDP-593,90-593_Site,McClymont et al. (2016),doi:10.1002/2016PA002954,Dudley and Nelson (1994),doi:10.1016/0377-8398(94)90008-6,A,A,-40.5078,167.6745,15.784540,128,1.166747,15.012721,0.771819
1,E45-29,E45-29,Howard and Prell (1992),doi:10.1029/91PA02994,Howard and Prell (1992),doi:10.1029/91PA02994,A,F,-44.8800,-106.5200,10.123284,128,1.034369,10.866542,-0.743258
2,E49-17,E49-17,Howard and Prell (1992),doi:10.1029/91PA02994,Howard and Prell (1992),doi:10.1029/91PA02994,A,F,-48.2833,90.2500,7.513381,128,3.764598,6.217212,1.296169
3,E49-18,E49-18,Howard and Prell (1992),doi:10.1029/91PA02994,Howard and Prell (1992),doi:10.1029/91PA02994,A,F,-46.0500,90.1500,9.739070,128,-0.254767,7.812882,1.926188
4,E49-19,E49-19,Rickaby and Elderfield (1999),doi:10.1029/1999PA900007,Rickaby and Elderfield (1999),doi:10.1029/1999PA900007,A,M,-43.8870,90.1000,10.658008,128,2.639316,10.123778,0.534229
5,FR1-94-GC3,FR1/94-GC3,De Deckker et al. (2019),doi:10.1080/08120099.2018.1495101,De Deckker et al. (2019),doi:10.1080/08120099.2018.1495101,A,F,-44.2600,150.0000,13.875218,128,0.171558,12.325758,1.549460
6,GeoB3227-5,GeoB3327-5,Ho et al. (2012),doi:10.1029/2012PA002317,Ho et al. (2012),doi:10.1029/2012PA002317,A,A,-43.2400,-79.9917,12.362086,128,1.233014,11.504871,0.857215
7,KH94-4,KH94-4TSP-2PC,Ikehara et al. (1997),doi:10.1029/97GL00429,Ikehara et al. (1997),doi:10.1029/97GL00429,A,A,-48.1400,146.8700,10.345097,128,3.204270,9.379837,0.965260
8,MD02-2488,MD02-2488,Govin et al. (2009),doi:10.1029/2008PA001603,Govin et al. (2009),doi:10.1029/2008PA001603,A,F,-46.4800,88.0200,8.903478,128,0.891012,7.925800,0.977678
9,MD02-2588,MD02-2588,Romero et al. (2015),doi:10.1002/2014PA002636,Starr et al. (2020),doi:10.1594/PANGAEA.921315,A,A,-41.2000,25.5000,15.344190,128,4.445560,14.120686,1.223505


In [6]:
lig_recs['MC']['interpolated']

,Station,lat,lon,AICC age [ka],rec_sic_sep,rec_sst_jfm,hadisst_sic_sep,hadisst_sst_jfm,sic_anom_hadisst_sep,sst_anom_hadisst_jfm
0,ANTA91-8,-70.78,172.83,127,42.828880,0.955085,94.000012,-0.426684,-51.171131,1.381768
1,ELT17-9,-63.08,-135.12,127,11.840205,2.838963,32.999995,0.948613,-21.159790,1.890349
2,MD03-2603,-64.28,139.38,127,34.145527,1.650259,79.999983,0.294749,-45.854456,1.355510
3,NBP9802-04,-64.20,-170.08,127,16.287606,1.987311,81.000000,0.752920,-64.712395,1.234391
4,PC509,-68.31,-86.03,127,37.516742,1.019577,92.000002,0.781203,-54.483259,0.238375
5,TPC287,-60.31,-36.65,127,56.137360,0.738042,84.000009,0.151935,-27.862649,0.586107
6,TPC288,-59.14,-37.96,127,29.768624,2.386726,79.000026,0.591002,-49.231402,1.795724
7,TPC290,-55.55,-45.02,127,26.199359,3.226982,15.000005,3.896293,11.199354,-0.669312
8,U1361A,-64.41,143.89,127,9.060829,3.989584,77.000028,0.650832,-67.939199,3.338752


# Reconstruction of DC

In [3]:
# print(lig_recs['DC']['annual_128'].shape)
# print(lig_recs['DC']['JFM_128'].shape)
# len(lig_recs['DC']['annual_128'].Station.unique())
lig_recs['DC']['annual_128']

,Station,Event,Reference (SST proxy reference),URL ref (SST proxy reference),Reference (d18O reference),URL ref (d18O reference),"Season (A annual, S summer (months JFM))","Proxy (A alkenone UK'37, D diatom as...)",Latitude,Longitude,SST [°C] (Reconstructed),Age [ka BP] (Chronology follows Lisiecki a...),SST anomaly [°C] (Anomaly relative to the World...),HadISST_annual,sst_anom_hadisst_ann
0,DSDP-593,90-593_Site,McClymont et al. (2016),doi:10.1002/2016PA002954,Dudley and Nelson (1994),doi:10.1016/0377-8398(94)90008-6,A,A,-40.5078,167.6745,15.784540,128,1.166747,15.012721,0.771819
1,E45-29,E45-29,Howard and Prell (1992),doi:10.1029/91PA02994,Howard and Prell (1992),doi:10.1029/91PA02994,A,F,-44.8800,-106.5200,10.123284,128,1.034369,10.866542,-0.743258
2,E49-17,E49-17,Howard and Prell (1992),doi:10.1029/91PA02994,Howard and Prell (1992),doi:10.1029/91PA02994,A,F,-48.2833,90.2500,7.513381,128,3.764598,6.217212,1.296169
3,E49-18,E49-18,Howard and Prell (1992),doi:10.1029/91PA02994,Howard and Prell (1992),doi:10.1029/91PA02994,A,F,-46.0500,90.1500,9.739070,128,-0.254767,7.812882,1.926188
4,E49-19,E49-19,Rickaby and Elderfield (1999),doi:10.1029/1999PA900007,Rickaby and Elderfield (1999),doi:10.1029/1999PA900007,A,M,-43.8870,90.1000,10.658008,128,2.639316,10.123778,0.534229
5,FR1-94-GC3,FR1/94-GC3,De Deckker et al. (2019),doi:10.1080/08120099.2018.1495101,De Deckker et al. (2019),doi:10.1080/08120099.2018.1495101,A,F,-44.2600,150.0000,13.875218,128,0.171558,12.325758,1.549460
6,GeoB3227-5,GeoB3327-5,Ho et al. (2012),doi:10.1029/2012PA002317,Ho et al. (2012),doi:10.1029/2012PA002317,A,A,-43.2400,-79.9917,12.362086,128,1.233014,11.504871,0.857215
7,KH94-4,KH94-4TSP-2PC,Ikehara et al. (1997),doi:10.1029/97GL00429,Ikehara et al. (1997),doi:10.1029/97GL00429,A,A,-48.1400,146.8700,10.345097,128,3.204270,9.379837,0.965260
8,MD02-2488,MD02-2488,Govin et al. (2009),doi:10.1029/2008PA001603,Govin et al. (2009),doi:10.1029/2008PA001603,A,F,-46.4800,88.0200,8.903478,128,0.891012,7.925800,0.977678
9,MD02-2588,MD02-2588,Romero et al. (2015),doi:10.1002/2014PA002636,Starr et al. (2020),doi:10.1594/PANGAEA.921315,A,A,-41.2000,25.5000,15.344190,128,4.445560,14.120686,1.223505


In [4]:
lig_recs['DC']['JFM_128']

,Station,Event,Reference (SST proxy reference),URL ref (SST proxy reference),Reference (d18O reference),URL ref (d18O reference),"Season (A annual, S summer (months JFM))","Proxy (A alkenone UK'37, D diatom as...)",Latitude,Longitude,SST [°C] (Reconstructed),Age [ka BP] (Chronology follows Lisiecki a...),SST anomaly [°C] (Anomaly relative to the World...),HadISST_JFM,sst_anom_hadisst_jfm
0,DSDP-593,90-593_Site,McClymont et al. (2016),doi:10.1002/2016PA002954,Dudley and Nelson (1994),doi:10.1016/0377-8398(94)90008-6,S,A,-40.5078,167.6745,18.164817,128,1.318002,17.628258,0.536558
1,E45-29,E45-29,Howard and Prell (1992),doi:10.1029/91PA02994,Howard and Prell (1992),doi:10.1029/91PA02994,S,F,-44.8800,-106.5200,11.123017,128,2.002886,12.900438,-1.777422
2,E49-17,E49-17,Howard and Prell (1992),doi:10.1029/91PA02994,Howard and Prell (1992),doi:10.1029/91PA02994,S,F,-48.2833,90.2500,8.175265,128,4.783591,7.093046,1.082219
3,E49-18,E49-18,Howard and Prell (1992),doi:10.1029/91PA02994,Howard and Prell (1992),doi:10.1029/91PA02994,S,F,-46.0500,90.1500,11.675477,128,-0.344354,8.684106,2.991371
4,FR1-94-GC3,FR1/94-GC3,De Deckker et al. (2019),doi:10.1080/08120099.2018.1495101,De Deckker et al. (2019),doi:10.1080/08120099.2018.1495101,S,F,-44.2600,150.0000,15.722242,128,0.418220,14.188304,1.533937
5,GeoB3227-5,GeoB3327-5,Ho et al. (2012),doi:10.1029/2012PA002317,Ho et al. (2012),doi:10.1029/2012PA002317,S,A,-43.2400,-79.9917,14.905477,128,0.922223,13.822713,1.082764
6,KH94-4,KH94-4TSP-2PC,Ikehara et al. (1997),doi:10.1029/97GL00429,Ikehara et al. (1997),doi:10.1029/97GL00429,S,A,-48.1400,146.8700,11.461409,128,4.316484,10.517632,0.943777
7,MD02-2488,MD02-2488,Govin et al. (2009),doi:10.1029/2008PA001603,Govin et al. (2009),doi:10.1029/2008PA001603,S,F,-46.4800,88.0200,9.733049,128,1.664914,8.829524,0.903525
8,MD02-2588,MD02-2588,Romero et al. (2015),doi:10.1002/2014PA002636,Starr et al. (2020),doi:10.1594/PANGAEA.921315,S,A,-41.2000,25.5000,17.161237,128,5.412864,15.763505,1.397732
9,MD06-2986,MD06-2986,Hayward et al. (2012),doi:10.1016/j.marmicro.2011.10.003,Ronge et al. (2015),doi:10.1002/2014PA002727,S,F,-43.4485,167.9000,16.934843,128,1.427277,16.118504,0.816339


# Reconstruction of EC

In [11]:
lig_recs['EC']['original']
lig_recs['EC']['SO_ann']
lig_recs['EC']['SO_jfm']
lig_recs['EC']['NH_ann']
lig_recs['EC']['NH_sum']
lig_recs['EC']['GrIS_am']
lig_recs['EC']['AIS_am']

,Station,Latitude,Longitude,Elevation [m],Area,Temperature reconstruction method,Uncertainty on temperature reconstruction [°C],Type,References temperature data,Surface air temperature [°C],...,125 ka Median PIAn [°C],125 ka 2s PIAn [°C],120 ka Median PIAn [°C],120 ka 2s PIAn [°C],115 ka Median PIAn [°C],115 ka 2s PIAn [°C],127-130 ka diff [°C],127-130 ka diff 2s [°C],125-127 ka diff [°C],125-127 ka diff 2s [°C]
43,EDML,-75.00,0.00,2892,Antarctica,Water isotopes,1.5,Annual air temperature,"Masson-Delmotte et al., 2011",-44.6,...,0.79,2.89,-0.18,2.88,-1.54,2.89,-0.46,0.01,-0.13,0.01
44,EDC,-75.10,123.35,3233,Antarctica,Water isotopes,1.5,Annual air temperature,"Masson-Delmotte et al., 2011",-54.5,...,1.48,2.98,1.33,2.98,-1.57,2.98,0.88,0.01,-1.40,0.01
45,Dome F,-77.32,39.70,3810,Antarctica,Water isotopes,1.5,Annual air temperature,"Masson-Delmotte et al., 2011",-57.0,...,1.71,3.05,1.01,3.04,-2.11,3.05,0.43,0.13,-1.57,0.06
46,Vostok,-78.47,106.87,3488,Antarctica,Water isotopes,1.5,Annual air temperature,"Masson-Delmotte et al., 2011",-55.3,...,1.48,2.91,0.70,2.90,-2.17,2.91,0.46,0.01,-0.43,0.01


# Reconstruction of JH

In [7]:
lig_recs['JH']['original']
lig_recs['JH']['SO_ann']
lig_recs['JH']['SO_jfm']
lig_recs['JH']['NH_ann']
lig_recs['JH']['NH_jas']

,Station,Region,Latitude,Longitude,Type,HadISST1.1 (1880-1889) (°C),Interpolated 127 ka value (°C),127 ka SST anomaly (°C),127 ka 2σ (°C)
0,EW9302-JPC8,North Atlantic,61.00,-25.00,Summer SST,9.38,12.782804,3.402804,4.312480
1,ODP980,North Atlantic,55.80,-14.11,Summer SST,11.92,14.473200,2.553200,5.029987
2,NA27-25,North Atlantic,55.57,-14.75,Summer SST,11.92,15.285128,3.400000,2.896113
3,NEAP18K,North Atlantic,53.00,-30.00,Summer SST,10.90,15.697547,4.800000,3.667836
4,MD04-2845,North Atlantic,45.35,-5.22,Summer SST,15.04,17.805647,2.765647,8.285585
5,SU92-03,North Atlantic,43.20,-10.11,Summer SST,15.50,17.232561,1.732561,6.281933
6,CH69-K09,North Atlantic,41.76,-47.35,Summer SST,16.70,16.269891,-0.430109,4.321275
7,MD95-2040,North Atlantic,40.58,-9.86,Summer SST,16.29,14.055527,-2.234473,6.689011
8,SU90-03,North Atlantic,40.51,-32.05,Summer SST,19.23,20.953101,1.723101,4.590814


# Reconstruction of MC

In [4]:
lig_recs['MC'].keys()

dict_keys(['original', 'interpolated'])

In [5]:
lig_recs['MC']['original']

,Station,Latitude,Longitude,Depth (cmbsf),Age on AICC2012 chronology (ka),MAT September sea-ice concentration (%),MAT summer SST (degrees celsius)
0,ANTA91-8,-70.78,172.83,272.0,121.214074,38.62,1.08
1,ANTA91-8,-70.78,172.83,274.0,121.980200,33.33,0.99
2,ANTA91-8,-70.78,172.83,276.0,122.746326,38.15,0.96
3,ANTA91-8,-70.78,172.83,278.0,123.512452,34.66,1.04
4,ANTA91-8,-70.78,172.83,280.0,124.278578,49.13,0.75
...,...,...,...,...,...,...,...
140,U1361A,-64.41,143.89,211.0,129.261692,23.70,2.40
141,U1361A,-64.41,143.89,213.0,130.817010,76.40,0.40
142,U1361A,-64.41,143.89,217.0,133.284836,35.80,2.20
143,U1361A,-64.41,143.89,222.0,135.567196,53.10,0.70


In [6]:
lig_recs['MC']['interpolated']

,Station,lat,lon,AICC age [ka],rec_sic_sep,rec_sst_jfm,hadisst_sic_sep,hadisst_sst_jfm,sic_anom_hadisst_sep,sst_anom_hadisst_jfm
0,ANTA91-8,-70.78,172.83,127,42.828880,0.955085,94.000012,-0.426684,-51.171131,1.381768
1,ELT17-9,-63.08,-135.12,127,11.840205,2.838963,32.999995,0.948613,-21.159790,1.890349
2,MD03-2603,-64.28,139.38,127,34.145527,1.650259,79.999983,0.294749,-45.854456,1.355510
3,NBP9802-04,-64.20,-170.08,127,16.287606,1.987311,81.000000,0.752920,-64.712395,1.234391
4,PC509,-68.31,-86.03,127,37.516742,1.019577,92.000002,0.781203,-54.483259,0.238375
5,TPC287,-60.31,-36.65,127,56.137360,0.738042,84.000009,0.151935,-27.862649,0.586107
6,TPC288,-59.14,-37.96,127,29.768624,2.386726,79.000026,0.591002,-49.231402,1.795724
7,TPC290,-55.55,-45.02,127,26.199359,3.226982,15.000005,3.896293,11.199354,-0.669312
8,U1361A,-64.41,143.89,127,9.060829,3.989584,77.000028,0.650832,-67.939199,3.338752
